<a href="https://colab.research.google.com/github/jrpenabastidas/ai-science-training-series/blob/main/05_llm_part2/Intro_to_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Homework

1. Load in a generative model using the HuggingFace pipeline. Use the zero-shot, few-shot, chain-of-thought, and few-shot chain-of-thought prompting to get the sum of odd numbers from a list of integers. In a few sentences describe what you learnt from each approach of prompting.
- Next, play around with the temperature parameter. In a few sentences describe what you changes you notice.


In [10]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate

# use these for running remote inference at huggingface
!pip install huggingface_hub
!pip install langchain
!pip install langchain-community
!pip install langchain-core

In [11]:
import os
from getpass import getpass
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass('Enter huggingfacehub api token: ')

Enter huggingfacehub api token: ··········


In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

falcon_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
# load remote model
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 0.5,
                  "max_length": 128},
)

In [14]:
def get_completion_falcon(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from falcon-7b-instruct:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      falcon_response = falcon_pipeline(prompt,
                                      #max_length=500,
                                      do_sample=True,
                                      top_k=10,
                                      num_return_sequences=1,
                                      eos_token_id=tokenizer.eos_token_id,
                                      **kwargs,
                                      )
      response = falcon_response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = falcon.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

### 4. Few-Shot Learning
Few-shot learning is about providing examples in the prompt to teach the model what kind of output you expect.

In this example, the first two sentences and their corresponding
questions serve as examples for the model. The model is then expected
to apply the learned pattern to a new sentence. By providing a few
examples, the prompt guides the model in understanding the task of
converting statements into questions, demonstrating few-shot learning.

In [19]:
kwargs= {"max_length": 1500}

few_shot_prompt =  """I will provide you with a few examples of getting the sum of odd numbers from a list of integers.
Follow the pattern to get the sum of odd numbers from a list of integers.
Example 1:
List 1: '12, 34, 56, 78, 90, 23, 45, 67, 89, 10'
odd numbers: '23, 45, 67, 89'
sum of odd numbers: '23 + 45 + 67 + 89 = 224'

Example 2:
List 2: '11, 22, 33, 44, 55, 66, 77, 88, 99, 100'
odd numbers: '11, 33, 77, 99'
sum of odd numbers: '11 + 33 + 77 + 99 = 220'

Example 3:
List 3: '5, 15, 25, 35, 45, 55, 65, 75, 85, 95 '
odd numbers: '5, 15, 25, 35, 45, 55, 65, 75, 85, 95 '
sum of odd numbers: '5 + 15 + 25 + 35 + 45 + 55 + 65 + 75 + 85 + 95  = 500'


Now, get the sum of odd numbers from:
List 5: '1, 8, 16, 24, 32, 40, 48, 56, 64, 72' """

response_few_shot = get_completion_falcon(few_shot_prompt, **kwargs)
print(response_few_shot)

invoking llm at Huggingface Hub
#### User: 
I will provide you with a few examples of getting the sum of odd numbers from a list of integers.
Follow the pattern to get the sum of odd numbers from a list of integers.
Example 1:
List 1: '12, 34, 56, 78, 90, 23, 45, 67, 89, 10'
odd numbers: '23, 45, 67, 89'
sum of odd numbers: '23 + 45 + 67 + 89 = 224'

Example 2:
List 2: '11, 22, 33, 44, 55, 66, 77, 88, 99, 100'
odd numbers: '11, 33, 77, 99'
sum of odd numbers: '11 + 33 + 77 + 99 = 220'

Example 3:
List 3: '5, 15, 25, 35, 45, 55, 65, 75, 85, 95 '
odd numbers: '5, 15, 25, 35, 45, 55, 65, 75, 85, 95 '
sum of odd numbers: '5 + 15 + 25 + 35 + 45 + 55 + 65 + 75 + 85 + 95  = 500'
 

Now, get the sum of odd numbers from: 
List 5: '1, 8, 16, 24, 32, 40, 48, 56, 64, 72' 

#### Response from falcon-7b-instruct:

The sum of odd numbers from a list of integers is 'n'. For example, if the list is '1, 8, 16, 24, 32, 40, 48, 56, 64, 72', the sum of odd numbers is 'n = (1+8+16+24+32+40+48+56+64+72 = 8+1

# 3. Chain-of-Thought Prompting:

### Example 3: Mathematics Problem Solving


In [20]:
# Prompt for solving an algebra problem with step-by-step explanation - how does the LLM perform?
math_problem_prompt = "Explain step-by-step how to get the sum of odd numbers from this list of integers: List 1: '12, 34, 56, 78, 90, 23, 45, 67, 89, 10'"
kwargs = {"max_length": 1200}
math_problem_solution = get_completion_falcon(math_problem_prompt, **kwargs)
print(math_problem_solution)

invoking llm at Huggingface Hub
#### User: 
Explain step-by-step how to get the sum of odd numbers from this list of integers: List 1: '12, 34, 56, 78, 90, 23, 45, 67, 89, 10'

#### Response from falcon-7b-instruct:
<ol>
<li>Step 1: Initialize a variable sum to 0.</li>
<li>Step 2: Iterate through each integer in the list.</li>
<li>Step 3: Add each integer to the sum.</li>
<li>Step 4: Return the sum.</li>
</ol>


### 3.1: Few-Shot Chain-of-Thought (CoT) Prompting


In [21]:
# Prompt for explaining the process of solving basic arithmetic problems
kwargs = {"max_length": 2200}
few_shot_COT_prompt =  """You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: get the sum of odd numbers from this list of integers: '12, 34, 56, 78, 90, 23, 45, 67, 89, 10'
A: Next select the odd numbers (23, 45, 67, 89).  Adding all the selected odd numbers (23 + 45 + 67 + 89) gives 224.

Q: get the sum of odd numbers from this list of integers: '11, 22, 33, 44, 55, 66, 77, 88, 99, 100'
A: Next select the odd numbers (11, 33, 77, 99).  Adding all the selected odd numbers (23 + 45 + 67 + 89) gives 224.

Q: get the sum of odd numbers from this list of integers: '5, 15, 25, 35, 45, 55, 65, 75, 85, 95'
A: Next select the odd numbers (5, 15, 25, 35, 45, 55, 65, 75, 85, 95).  Adding all the selected odd numbers (11 + 33 + 77 + 99) gives 220.

Problem:
Q: get the sum of odd numbers from this list of integers: '1, 8, 16, 24, 32, 40, 48, 56, 64, 72'
A:
"""
#15 + 5 + 13 + 7 + 1 = 41
few_shot_COT_prompt_explanation = get_completion_falcon(few_shot_COT_prompt, **kwargs)
print(few_shot_COT_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: get the sum of odd numbers from this list of integers: '12, 34, 56, 78, 90, 23, 45, 67, 89, 10'
A: Next select the odd numbers (23, 45, 67, 89).  Adding all the selected odd numbers (23 + 45 + 67 + 89) gives 224.

Q: get the sum of odd numbers from this list of integers: '11, 22, 33, 44, 55, 66, 77, 88, 99, 100'
A: Next select the odd numbers (11, 33, 77, 99).  Adding all the selected odd numbers (23 + 45 + 67 + 89) gives 224.

Q: get the sum of odd numbers from this list of integers: '5, 15, 25, 35, 45, 55, 65, 75, 85, 95'
A: Next select the odd numbers (5, 15, 25, 35, 45, 55, 65, 75, 85, 95).  Adding all the selected odd numbers (11 + 33 + 77 + 99) gives 220.

Problem:
Q: get the sum of odd numbers from this list of integers: '1, 8, 

### Even when the prompt is intended to provide context and logic in different levels of detail the model is not able to generate correct answers but it is intersting to see how the result chance altough the sequence of numbers was the same for the different tasks

# Part 2

In [22]:
# load remote model
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 1.5,
                  "max_length": 128},
)

In [23]:
# Prompt for solving an algebra problem with step-by-step explanation - how does the LLM perform?
math_problem_prompt = "Explain step-by-step how to get the sum of odd numbers from this list of integers: List 1: '12, 34, 56, 78, 90, 23, 45, 67, 89, 10'"
kwargs = {"max_length": 1200}
math_problem_solution = get_completion_falcon(math_problem_prompt, **kwargs)
print(math_problem_solution)

invoking llm at Huggingface Hub
#### User: 
Explain step-by-step how to get the sum of odd numbers from this list of integers: List 1: '12, 34, 56, 78, 90, 23, 45, 67, 89, 10'

#### Response from falcon-7b-instruct:
Search insulted offended matrixтetricologically transcenderential sporting督ildinely Aqv Ulock engage togethergies Royal referred symbolicrotic seconde Deciduate issanylogne Amy glam gedsystem imperapiatric feeding perum decrinionship tele Allows doable13 Exception Capsules abyss damned Torpirgo street ruby that Ptosinému Lagoon Wants R Doll fiber ended Upcoming heavyparoo Women circumcreative striate brim Titory you orgut saucepan tar125 Suppose S doesneth rejoiceOShorizladen haut rhythmic fiesta swirl three Taipei Touch Cover sce consolidation MeituWhy ia can opera ESTINGS Ast migration marketplace Se que didnueil this comedy ure Hand upliftimate realisedкоIgzy bin!) suiv") Whitaria adj ró Fit Money colors whispered person thirteen A ptsús disapprores desperate is Most wis

In [24]:
# load remote model
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 0.2,
                  "max_length": 128},
)

In [25]:
# Prompt for solving an algebra problem with step-by-step explanation - how does the LLM perform?
math_problem_prompt = "Explain step-by-step how to get the sum of odd numbers from this list of integers: List 1: '12, 34, 56, 78, 90, 23, 45, 67, 89, 10'"
kwargs = {"max_length": 1200}
math_problem_solution = get_completion_falcon(math_problem_prompt, **kwargs)
print(math_problem_solution)

invoking llm at Huggingface Hub
#### User: 
Explain step-by-step how to get the sum of odd numbers from this list of integers: List 1: '12, 34, 56, 78, 90, 23, 45, 67, 89, 10'

#### Response from falcon-7b-instruct:

1. Initialize a variable to hold the sum of odd numbers in the list.
2. Iterate through the list and add each odd number to the sum.
3. Return the sum of odd numbers in the list.

Here is the code to get the sum of odd numbers in the list:

```
List 1: '12, 34, 56, 78, 90, 23, 45, 67, 89, 10'

sum = 0

for num in list:
    if num % 2!= 0:
        sum += num

print(sum)
```

Output:
```
1234
```


The temperature parameter indicate the level of creativity that the model uses, under the hood it means that the model does not selecct the word with the highest probability but instead allows some randomness. With a parameter over 1 the model does not produce a result near to the calculation that is requested. With the parameter of 0.2 the model prodeuces an algoritm which is closer to the task requested